# Data Literacy
#### University of Tübingen, Winter Term 2020/21
## Exercise Sheet 6
&copy; 2020 Prof. Dr. Philipp Hennig & Lukas Tatzel

This sheet is **due on Tuesday 15 December 2020 at 12noon sharp (i.e. before the start of the lecture).**

---

## Hypothesis Testing & *Hunting* for Significance 

**What is this week's tutorial about?** In this week's tutorial, we will analyse data from the 1. Fußball-Bundesliga (the German premier soccer league), reproducing and extending the analysis done in the lecture. The goal is to find out if there are teams that achieve significantly worse results this year than in previous years (possibly caused by empty stadiums etc. due to the Corona-pandemic). For that purpose we will conduct a hypothesis test. To be more precise, we will perform multiple tests, i.e. we will compute multiple $p$-values. We will also discuss whether and how we should therefore adapt our testing strategy. 

In [1]:
# Make inline plots vector graphics
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from IPython.display import set_matplotlib_formats

# Plotting setup
set_matplotlib_formats("pdf", "svg")
plt.rcParams["text.usetex"] = True
plt.rcParams["text.latex.preamble"] = r"\usepackage{amsfonts} \usepackage{amsmath}"

In this tutorial, we will use data provided by OpenLigaDB (https://www.openligadb.de/). The function `get_league_table` allows you to retrieve the data via the API interface and convert it into a Pandas dataframe.

In [2]:
import requests

API_ENDPOINT = "https://www.openligadb.de/api"

def get_league_table(league, year):
    """
    Get team rankings as Pandas dataframe.
    
    Parameters
    ----------
    league : str
        'bl1' for 1. Bundesliga, see https://github.com/OpenLigaDB/OpenLigaDB-Samples
    year : int
        Get data for this year
    """
    response = requests.get(f"{API_ENDPOINT}/getbltable/{league}/{year}")
    teams = response.json()
    return pd.DataFrame(teams)

# Get data for the 1. Bundesliga ('bl1') for 2020
data = get_league_table(league="bl1", year=2020)
display(data.head())

,TeamInfoId,TeamName,ShortName,TeamIconUrl,Points,OpponentGoals,Goals,Matches,Won,Lost,Draw,GoalDiff
0,40,FC Bayern München,Bayern,https://upload.wikimedia.org/wikipedia/commons...,23,16,34,10,7,1,2,18
1,6,Bayer Leverkusen,Leverkusen,https://upload.wikimedia.org/wikipedia/de/thum...,22,9,19,10,6,0,4,10
2,1635,RB Leipzig,Leipzig,https://upload.wikimedia.org/wikipedia/en/thum...,21,9,21,10,6,1,3,12
3,7,BV Borussia Dortmund 09,Dortmund,https://upload.wikimedia.org/wikipedia/commons...,19,10,22,10,6,3,1,12
4,131,VfL Wolfsburg,Wolfsburg,https://upload.wikimedia.org/wikipedia/commons...,18,10,16,10,4,0,6,6


**Task:** Your first task is to essentially recreate the following table from lecture 6. 

<img src="Table.PNG" alt="Drawing" style="width: 750px;"/>

**Recommended Steps:**
1. First, gather data for all years from 2010 to 2020 and store this data into a single Pandas dataframe. Each time you receive a dataframe from `get_league_table`, extract the relevant columns and add the corresponding year as a new column `"year"`.
2. Split this dataframe into two separate dataframes: One for the year 2020, the other one for everything before (i.e. 2010 to 2019). 
3. Merge the two dataframes from on the `"TeamName"`-column. Make sure that you only include teams that play in 2020 and played in at least one other season in the past. 

In [3]:
# Step 1: Gather data
data = get_league_table(league="bl1", year=2010)
data = data[['TeamName', 'Matches', 'Won', 'Lost', 'Draw']]
for i in range(2011, 2020):
    temp = get_league_table(league="bl1", year=i)
    temp = temp[['TeamName', 'Matches', 'Won', 'Lost', 'Draw']]
    data = data.append(temp, ignore_index=True)
data = data.groupby(["TeamName"]).sum()
display(data)

,Matches,Won,Lost,Draw
TeamName,,,,
1. FC Kaiserslautern,68,17,33,18
1. FC Köln,238,67,108,63
1. FC Nürnberg,170,44,80,46
1. FC Union Berlin,34,12,17,5
1. FSV Mainz 05,340,118,141,81
BV Borussia Dortmund 09,340,203,65,72
Bayer Leverkusen,340,171,96,73
Borussia Mönchengladbach,340,152,113,75
Eintracht Braunschweig,34,6,21,7


In [4]:
# Step 2: Split dataframe
data = data.rename(columns = {'Matches':'MatchesPre2020', 'Won':'WonPre2020', 'Lost':'LostPre2020', 'Draw':'DrawPre2020'})

In [5]:
# Step 3: Merge dataframes
data_2020 = get_league_table(league="bl1", year=2020)[["Matches", "TeamName", "Won", "Lost", "Draw"]]
data = pd.merge(data_2020, data, how='inner', on='TeamName')
data = data.rename(columns = {'Matches':'Matches2020', "Won": "Won2020", "Lost": "Lost2020", "Draw": "Draw2020"})
display(data.head())

,Matches2020,TeamName,Won2020,Lost2020,Draw2020,MatchesPre2020,WonPre2020,LostPre2020,DrawPre2020
0,10,FC Bayern München,7,1,2,340,255,38,47
1,10,Bayer Leverkusen,6,0,4,340,171,96,73
2,10,RB Leipzig,6,1,3,136,72,28,36
3,10,BV Borussia Dortmund 09,6,3,1,340,203,65,72
4,10,VfL Wolfsburg,4,0,6,340,127,121,92


Our goal is to find out if there are teams that achieve significantly worse results this year than in previous years. In the lecture, you already learned about a statistical test for this purpose: 

- First, we put a beta-prior on $f_0$ (the winning probability before 2020) which is based on $m_0$ (the number of wins before 2020) in $n_0$ matches (the number of matches before 2020).

- Under the null hypothesis $H_0: f_1 = f_0$, the number of wins in 2020 $m_1$ (given the number of matches in 2020 $n_1$) follows a binomial distribution. 

- Putting these building blocks together, we obtain a [beta-binomial distribution](https://en.wikipedia.org/wiki/Beta-binomial_distribution) for the *evidence* $p(m_1 \vert n_1, m_0, n_0)$. This tells us the probability to oberserve $m_1$ wins in 2020, given the number of matches in 2020 $n_1$ and the statistics $m_0$, $n_0$ for the years before. 

- The $p$-value represents the probability to observe a certain number of wins or *more extreme* results. Lets assume, the team we consider, has won $3$ times this season so far. Since we are interested in teams that have played particularly badly this year, we sum $p(m_1 \vert n_1, m_0, n_0)$ over $m_1 = 0, 1, 2, 3$. Evaluating the cumulative distribution function `betabinom.cdf` performs this summation for us. 

- If this probability is small, then it is very unlikely that the oberved data has been generated from the winning probability $f_0$. Or, in other woirds, the team has played particularly badly so far. We thus reject $H_0$ if $p \leq \alpha := 5 \%$. 

**Task:** Compute the $p$-values for every team and add the results as a new column `"p-value (won)"`. Check if there are teams whose $p$-value falls below the $5 \%$ threshold. 

In [6]:
from scipy.stats import betabinom

def p_val_won(m_1, n_1, m_0, n_0):
    """
    Compute p-value by summing the evidence p(m_1 | n_1, m_0, n_0) over the 
    observed number of wins and 'more extreme' (i.e. smaller) results.
    
    Parameters
    ----------
    m_1 : int
        Number of wins in 2020 (0 <= m_1 <= n_1)
    n_1 : int
        Number of matches in 2020 (n_1 > 0)
    m_0 : int
        Number of wins before 2020 (0 <= m_0 <= n_0)
    n_0 : int
        Number of matches before 2020 (n_0 > 0)
    """
    return betabinom.cdf(m_1, n_1, m_0 + 1, n_0 - m_0 + 1)

In [7]:
# Compute p-values, filter for significant results
data["p_value_won"] = p_val_won(data["Won2020"], data["Matches2020"], data["WonPre2020"], data["MatchesPre2020"])
display(data)
print("Significant values:")
display(data[data["p_value_won"] < 0.05])

,Matches2020,TeamName,Won2020,Lost2020,Draw2020,MatchesPre2020,WonPre2020,LostPre2020,DrawPre2020,p_value_won
0,10,FC Bayern München,7,1,2,340,255,38,47,0.477655
1,10,Bayer Leverkusen,6,0,4,340,171,96,73,0.819741
2,10,RB Leipzig,6,1,3,136,72,28,36,0.768156
3,10,BV Borussia Dortmund 09,6,3,1,340,203,65,72,0.624228
4,10,VfL Wolfsburg,4,0,6,340,127,121,92,0.693864
5,10,1. FC Union Berlin,4,2,4,34,12,17,5,0.706865
6,10,Borussia Mönchengladbach,4,2,4,340,152,113,75,0.511349
7,10,VfB Stuttgart,3,2,5,272,87,128,57,0.591938
8,10,Eintracht Frankfurt,2,1,7,306,105,126,75,0.278324
9,10,TSG 1899 Hoffenheim,3,4,3,340,120,120,100,0.504235


Significant values:


,Matches2020,TeamName,Won2020,Lost2020,Draw2020,MatchesPre2020,WonPre2020,LostPre2020,DrawPre2020,p_value_won
16,10,FC Schalke 04,0,7,3,340,140,119,81,0.00538


Our goal is to find teams that play significantly worse this year compared to previous years. We did this by checking whether the number of games won this year is surprisingly low. Now, we will use an alternative approach: We check whether the number of games *lost* this year is particularly *high*. For this, we can re-use the statistical beta-binomial model from above by simply plugging in the number of *lost* games for $m_0$ and $m_1$. 

**Task:** Compute the corresponding $p$-values and store the results in a new column `"p-value (lost)`. Note that you cannot simply use the `p_val_won` fuction from above. This time, the question is: What is the probability for oberving $m_1$ or *more* lost matches.

In [8]:
def p_val_lost(m_1, n_1, m_0, n_0):
    """
    Compute p-value by ...
    
    Parameters
    ----------
    m_1 : int
        Number of lost matches in 2020 (0 <= m_1 <= n_1)
    n_1 : int
        Number of matches in 2020 (n_1 > 0)
    m_0 : int
        Number of lost matches before 2020 (0 <= m_0 <= n_0)
    n_0 : int
        Number of matches before 2020 (n_0 > 0)
    """
    return betabinom.cdf(m_1, n_1, 1 - n_0 - m_0, 1 - m_0)

In [9]:
# Compute p-values, filter for significant results
data["p_value_lost"] = p_val_lost(data["Lost2020"], data["Matches2020"], data["LostPre2020"], data["MatchesPre2020"])
display(data)
print("Significant values:")
display(data[data["p_value_lost"] < 0.05])

,Matches2020,TeamName,Won2020,Lost2020,Draw2020,MatchesPre2020,WonPre2020,LostPre2020,DrawPre2020,p_value_won,p_value_lost
0,10,FC Bayern München,7,1,2,340,255,38,47,0.477655,NaN
1,10,Bayer Leverkusen,6,0,4,340,171,96,73,0.819741,NaN
2,10,RB Leipzig,6,1,3,136,72,28,36,0.768156,NaN
3,10,BV Borussia Dortmund 09,6,3,1,340,203,65,72,0.624228,NaN
4,10,VfL Wolfsburg,4,0,6,340,127,121,92,0.693864,NaN
5,10,1. FC Union Berlin,4,2,4,34,12,17,5,0.706865,NaN
6,10,Borussia Mönchengladbach,4,2,4,340,152,113,75,0.511349,NaN
7,10,VfB Stuttgart,3,2,5,272,87,128,57,0.591938,NaN
8,10,Eintracht Frankfurt,2,1,7,306,105,126,75,0.278324,NaN
9,10,TSG 1899 Hoffenheim,3,4,3,340,120,120,100,0.504235,NaN


Significant values:


,Matches2020,TeamName,Won2020,Lost2020,Draw2020,MatchesPre2020,WonPre2020,LostPre2020,DrawPre2020,p_value_won,p_value_lost


By now, we conducted $2 \cdot 17 = 34$ hypothesis tests at significance level $\alpha = 5 \%$. $\alpha$ corresponds to the probability of a type I error, i.e. rejecting the null hypothesis given that it is true. However, the more tests we perform, the higher the chance of observing a rare event simply due to chance. For example, if we assume that $H_0$ holds for every team, the chance of falsely rejecting at least one out of $34$ hypothesis is $1 - (1-0.05)^{34} \approx 83 \%$. Thus, it is quite likely that one of the results we found is a type I error. The Bonferroni correction is one possibility for compensating for that effect by decreasing the significance level. The significance level is defined as the original one divided by the total number of hypothesis. 

**Task:** Define the new significance level and see whether you can (still) find significant results. 

In [10]:
# Filter for significant results (Bonferroni correction)
display(data[(data["p_value_won"] < 0.05 / 34) | (data["p_value_lost"] < 0.05 / 34)])

,Matches2020,TeamName,Won2020,Lost2020,Draw2020,MatchesPre2020,WonPre2020,LostPre2020,DrawPre2020,p_value_won,p_value_lost


The Bonferroni method tends to be too *conservative*, i.e. the significance level might be too restrictive. This is especially the case when the tests are dependent. 

**Task:** Think about if the tests we performed are dependent or independent and give a short explanation.

**Solution:** 

We conducted multiple hypothesis tests at significance level $\alpha = 5 \%$. $\alpha$ corresponds to the probability of a type I error, i.e. $\alpha = P(\text{type 1 error}) = P(p \leq \alpha\,|\,H_0\,\text{ is true}) = \text{cdf}_p(\alpha)$. So, under $H_0$, the cumulative distribution function of the $p$-value at $\alpha$ is $\text{cdf}_p(\alpha) = \alpha$. That implies that $p$ is uniformly distributed under $H_0$. Let us visually explore, if the observed $p$-values are likely to be drawn from a uniform distribution. This can be done by a so-called Q-Q plot. 

The idea of a Q-Q Plot is to compare the empirical quantiles of the empirical distribution of the $p$-values with the quantiles of the theoretical distribution (the uniform distribution as explained above). Let $\beta \in (0, 1)$. 
- The theoretical $\beta$-quantile of the uniform distribution is $q_{\beta} = \beta$.
- The empirical $\beta$-quantile of the *ascendingly ordered* sample $(p_1, ..., p_n)$ is $q_{\beta} = p_{\lfloor n\cdot\beta + 1\rfloor}$

**Task:** Complement the two functions below such that they return the quantiles defined above. Do not use `numpy.quantile` or similar.  Generate a vector that discretizes the variable $\beta$ and compute the corresponding quantiles. Plot the theoretical quantiles over the empirical quantiles. If the distributions are *similar*, the points will lie on the $45°$ line $y = x$. The result should look like this: 

<img src="Plot.PNG" alt="Drawing" style="width: 350px;"/>

In [11]:
from math import floor

def q_theoretical(beta):
    """
    Compute theoretical beta-quantile of uniform distribution.

    Parameters
    ----------
    beta : array-like, shape=(n,)
    """
    raise NotImplementedError

def q_empirical(beta, p):
    """
    Compute empirical beta-quantile of sample p.

    Parameters
    ----------
    beta : array-like, shape=(n,)
    p : array-like, shape=(n,)
        Unordered sample
    """
    raise NotImplementedError

In [12]:
# Create Q-Q Plot


Of course, the two generated lines do not coincide perfectly with the $45°$ line $y = x$. That raises the question, what deviation from that line we would expect if the $p$-values were actually drawn from a uniform distribution. One way to approach this question visually is to generate multiple samples of a uniform distribution (each sample consisting of $17$ numbers, like `p-values (won)` and `p-values (lost)`) and make a Q-Q Plot for each sample. So, we basically sample Q-Q plots under $H_0$. 

**Task:** Generate multiple (e.g. $100$) samples as desribed above and show the corresponding Q-Q Plots together with the two lines from the previous task. 

In [13]:
# Create Q-Q Plot
